# Using Python to Reverse-Engineer Guitar Effects
Author: Dan Lynch\
Date: November 21, 2020

---

This notebook will demonstrate some concepts in signal processing and some signal processing/analysis tools available in Python, primarily SciPy.

We'll use a "simple" example: distortion, in the style of the infamous Boss MT-2 Metal Zone pedal.
Although nonlinear, distortion is a little easier to demonstrate than delay-based effects such as echo, reverb, chorus, flanger, and phaser.

There are a number of Python modules for working with audio files; see [here](https://www.audiolabs-erlangen.de/resources/MIR/FMP/B/B_PythonAudio.html) for a helpful list. You'll see SciPy in that list, but the authors note that it doesn't always work well, and I have found that too. Instead, I'm using a module named [SoundFile](https://pysoundfile.readthedocs.io/en/0.9.0/#) that has a clean API. First we need to install it:

In [ ]:
!pip install soundfile

## Load Audio Files
This tutorial starts with two WAV files:
* `riff_clean.wav`: an unprocessed signal, recorded from my guitar direct to my computer
* `riff_dist.wav`: a heavily-distorted signal, recorded from my guitar passed through a Digitech RP-70 modeling pedal. This pedal is an all-in-one device that emulates a rack-full of guitar effects, including various amplifiers, compressors, EQs, chorus/flanger/phaser, and echo/reverb.

Here's the signal chain:
<img src="https://raw.githubusercontent.com/dlynch7/reverse_engineer_guitar_fx/main/distortion/recording_sig_chain.png" alt="recording_signal_chain" width="500"/>

In [ ]:
from IPython.display import Audio

# get the clean riff WAV:
riff_clean_url = 'https://raw.githubusercontent.com/dlynch7/reverse_engineer_guitar_fx/main/distortion/riff_clean.wav'

Audio(url=riff_clean_url)

In [ ]:
# get the distorted riff WAV:
riff_dist_url = 'https://raw.githubusercontent.com/dlynch7/reverse_engineer_guitar_fx/main/distortion/riff_dist.wav'

Audio(url=riff_dist_url) # it's a lot louder than the clean recording, so don't destroy your ears!

We're going to be doing a lot with these audio clips, so let's download them, temporarily, to our local workspace.

In [ ]:
import requests

r = requests.get(riff_clean_url, allow_redirects=True)
open('riff_clean.wav', 'wb').write(r.content) # 'wb' because we are writing a *binary* file

r = requests.get(riff_dist_url, allow_redirects=True)
open('riff_dist.wav', 'wb').write(r.content) # 'wb' because we are writing a *binary* file

Open your Colab notebook's filesystem by clicking on the folder icon to the left. You should see these files now. To see their sizes, mouse over them: both should be around 2 MB.

These files are "large" because they are WAV files, and WAV is one of the _**uncompressed**_ audio formats. In contrast, MP3s are smaller because they are compressed audio formats. Compression removes some audio information, and we want all the information available as we proceed to analyze and modify these signals, so that's why we're using WAV instead of MP3.

One more thing about the format: I recorded to 16-bit WAV files; 16-bit is typical CD-quality audio. You can record to 24-bit and 32-bit too (audio professionals use these resolutions), but those files are bigger, and it's hard to hear the difference. 16-bit is good enough for our purposes. The important thing is that the audio data is uncompressed.

Google Colab wasn't designed with audio processing in mind, and I'm unable to autosave my notebook once I download those two audio files. To get around this, execute the following cells in order.

In [ ]:
from scipy.io import wavfile
import scipy.io
import soundfile as sf

riff_clean_data, riff_clean_samplerate = sf.read('riff_clean.wav')
riff_clean_dict = {'wav_data':riff_clean_data,
                   'samplerate':riff_clean_samplerate,
                   'wav_fname':'riff_clean.wav'}
print(riff_clean_dict)

riff_dist_data, riff_dist_samplerate = sf.read('riff_dist.wav')
riff_dist_dict = {'wav_data':riff_dist_data,
                  'samplerate':riff_dist_samplerate,
                  'wav_fname':'riff_dist.wav'}
print(riff_dist_dict)

In [ ]:
# !rm riff_clean.wav

In [ ]:
# !rm riff_dist.wav

## Visualize each waveform

In [ ]:
# code based on 
# 1) https://docs.scipy.org/doc/scipy/reference/generated/scipy.io.wavfile.read.html
# 2) https://matplotlib.org/3.3.3/gallery/lines_bars_and_markers/cohere.html#sphx-glr-gallery-lines-bars-and-markers-cohere-py

import matplotlib.pyplot as plt
import matplotlib
import numpy as np

matplotlib.rcParams['font.size'] = 16.0

def plot_wave(wav_dict):
  wav_data = wav_dict['wav_data']
  samplerate = wav_dict['samplerate']

  # print(f"number of channels = {wav_data.shape[1]}")
  length = wav_data.shape[0] / samplerate
  # print(f"length = {length}s")
  time = np.linspace(0., length, wav_data.shape[0])
  
  fig, axs = plt.subplots(wav_data.shape[1],1,figsize=(20, 10))
  for channel in range(wav_data.shape[1]):
    axs[channel].plot(time, wav_data[:, channel])
    axs[channel].set_ylabel("Channel %d Amplitude" % channel)
    axs[channel].grid(True)
  axs[-1].set_xlabel("Time [s]")
  axs[0].set_title(wav_dict['wav_fname'])
  plt.show()
  return

plot_wave(riff_clean_dict)
plot_wave(riff_dist_dict)

Interesting! We can make an interesting observation already:

Notice that the distorted riff has near-constant amplitude, while the clean riff only has loud transients corresponding to each note I picked on the guitar.
So why the difference in perceived loudness? It's in our heads (a psychoacoustic effect): our ears/brains don't perceive instantaneous loudness but rather average loudness over some time window.
This explains why the distorted riff sounds much louder than the clean riff, even though they have almost identical peak amplitudes.

As an aside, this psychoacoustic effect led to widespread use (some might say abuse) of an audio effect called "limiting" which makes quiet parts louder. This effect was a key player in the "[Loudness War](https://en.wikipedia.org/wiki/Loudness_war)" of the 2000s.

## How does distortion work?
Let's zoom in near the end of the riff and compare the clean and distorted signals.

In [ ]:
def overlay_wavs(wav_dict_list,t_start,t_end):
  if not type(wav_dict_list) is list:
    raise TypeError('type(wav_dict_list) must be list.')
  if wav_dict_list == []:
    raise ValueError('wav_dict_list must have at least one element.')
  if not type(wav_dict_list[0]) is dict:
    raise TypeError('elements of wav_dict_list must be of type dict.') # could be more robust than it is
  if not (type(t_start) is float or type(t_start) is int):
    raise TypeError('type(t_start) must be float or int.')
  if not (type(t_end) is float or type(t_end) is int):
    raise TypeError('type(t_end) must be float or int.')

  if not t_start > 0:
    raise ValueError('t_start must be > 0')
  if not t_end > t_start:
    raise ValueError('t_start must be < t_end')

  # create a local dictionary to contain additional info:
  wavs_data = []
  for i in range(len(wav_dict_list)):
    length = wav_dict_list[i]['wav_data'].shape[0] / wav_dict_list[i]['samplerate']
    if not t_end < length:
      raise ValueError('t_end must be < %f' % length)
    time = np.linspace(0., length, wav_dict_list[i]['wav_data'].shape[0])
    wavs_data.append({'fname':      wav_dict_list[i]['wav_fname'],
                      'samplerate': wav_dict_list[i]['samplerate'],
                      'data':       wav_dict_list[i]['wav_data'],
                      'length':     length,
                      'time':       time})

  fig, axs = plt.subplots(len(wav_dict_list),1,figsize=(20, 10))
  for i in range(len(wavs_data)):
    n_start = int(t_start*wavs_data[i]['samplerate'])
    n_end = int(t_end*wavs_data[i]['samplerate'])
    t_range = np.linspace(t_start, t_end, n_end - n_start)
    axs[i].plot(t_range, wavs_data[i]['data'][n_start:n_end, 0]) # looking at channel 0, i.e., the left channel
    axs[i].set_ylabel("Channel 0 Amplitude, %s" % wavs_data[i]['fname'])
    axs[i].grid(True)
  axs[-1].set_xlabel("Time [s]")
  axs[0].set_title('Comparison of signals for %f <= time <= %f' % (t_start,t_end))
  plt.show()
  return

overlay_wavs([riff_clean_dict,riff_dist_dict],10.85,12.3)
overlay_wavs([riff_clean_dict,riff_dist_dict],10.85,11.0)
overlay_wavs([riff_clean_dict,riff_dist_dict],10.85,10.9)

Bear in mind that the upper signal is clean and the lower signal is distorted.
Let's take stock of what we can see after three rounds of zooming in on the signal:
* the distorted signal is louder;
* the distorted signal has more high-frequency content than the clean signal. You can see this pretty clearly around 10.86 seconds;
* the amplitude of the clean signal appears to exponentially decay after each transient, and if you imagine a vibrating string (and, indeed, the partial differential equation that models a vibrating string - the "wave equation"), this makes physical sense;
* the decay envelope of the distorted signal is "concave down", whereas the decay envelope of the clean signal is "concave up". This is noticeable at 10.85s, 10.865s, 10.88s, etc. This doesn't make physical sense!



### Overdrive and clipping
What's going on with the distorted signal might make more sense when you think about how guitar distortion works: _**clipping**_.

Most distortion effects are based on two stages:
* a gain stage (often achieved with a vacuum tube, a transistor, or an op-amp) that amplifies an incoming signal _a lot_.
* a clipping stage (sometimes built into the gain stage, other times achieved separately with a pair of diodes) that doesn't let the amplitude exceed some threshold.

Together, they act on an incoming signal thusly:
![proco_rat_clipping](https://www.electrosmash.com/images/tech/pro-co-rat/pro-co-rat-clipped-waveform.png)

The graph above comes from a [teardown and analysis of the ProCo RAT pedal](https://www.electrosmash.com/proco-rat) that does a great job explaining how this effect can be achieved using discrete electronic components.




### What does clipping *sound* like?
We just saw what clipping *looks* like, when applied to a pure sine wave, but what does it sound like? To answer that, let's mock up a clipping stage in Python, and let's implement it as a function that acts on NumPy arrays, since we can convert WAV files to NumPy arrays.

### Software clipping, attempt 1: hard (square) clipping
This function will need to take at least two inputs:
* an incoming signal, `sig_in`, which will be a NumPy array
* a threshold amplitude, `clip_thresh`, above which clipping will happen. Let's use a float between 0 and 1.

We'll rescale the clipping threshold to 15-bit range (0 to 2^15 - 1) inside the function. We're working with 16-bit audio, and audio is an AC signal, with positive and negative amplitude, so the maximum amplitude is 2^15 - 1 and the minimum amplitude is -2^15.

In [ ]:
def hard_clipper(sig_in,clip_thresh):
  if not type(sig_in) is np.ndarray:
    raise TypeError('type(sig_in) must be numpy.ndarray')
  if not (type(clip_thresh) is float or type(clip_thresh) is int):
    raise TypeError('type(clip_thresh) must be float or int')
  if (clip_thresh > 1 or clip_thresh < 0):
    raise ValueError('clip_thresh must be between 0 and 1.')

  sig_out = sig_in.copy()
  pos_thresh = clip_thresh
  neg_thresh = -clip_thresh

  # clip the positive part of sig_in:
  pos_clip_indices = sig_in > pos_thresh
  sig_out[pos_clip_indices] = pos_thresh

  # clip the negative part of sig_in:
  neg_clip_indices = sig_in < neg_thresh
  sig_out[neg_clip_indices] = neg_thresh

  return sig_out

samplerate = 44100; fs = 5 # too low to hear, but easy to see
t = np.linspace(0., 1., samplerate)
amplitude = 1
sine_data = amplitude * np.sin(2. * np.pi * fs * t)

hard_clip_75 = hard_clipper(sine_data,0.75)
hard_clip_50 = hard_clipper(sine_data,0.50)
hard_clip_25 = hard_clipper(sine_data,0.25)

fig, ax = plt.subplots(1,1,figsize=(20, 10))
ax.plot(t, sine_data,label='thresh = 1.0')
ax.plot(t, hard_clip_75,label='thresh = 0.75')
ax.plot(t, hard_clip_50,label='thresh = 0.50')
ax.plot(t, hard_clip_25,label='thresh = 0.25')

ax.set_ylabel("Amplitude")
ax.grid(True)
ax.legend()
ax.set_xlabel("Time [s]")
plt.show()

### Making up lost amplitude
Clipping is an effective way to introduce distortion, but it also makes everything quieter, and that isn't very fun. To fix that, most distortion effects have another gain stage after the clipping stage.

In Python, it's even easier: if we know our clipping threshold, we also know the amount of make-up gain we need to apply:
```python
makeup_gain = 1/clip_thresh
```
* if `clip_thresh` = 1, `makeup_gain` = 1
* if `clip_thresh` = 0.5, `makeup_gaint` = 2
* if `clip_thresh` = 0.1, `makeup_gain` = 10

Let's include this makeup gain in our hard-clipping function:

In [ ]:
def hard_clipper_makeup(sig_in,clip_thresh):
  if not type(sig_in) is np.ndarray:
    raise TypeError('type(sig_in) must be numpy.ndarray')
  if not (type(clip_thresh) is float or type(clip_thresh) is int):
    raise TypeError('type(clip_thresh) must be float or int')
  if (clip_thresh > 1 or clip_thresh < 0):
    raise ValueError('clip_thresh must be between 0 and 1.')

  sig_out = sig_in.copy()

  # clip the positive part of sig_in:
  pos_clip_indices = sig_in > clip_thresh
  sig_out[pos_clip_indices] = clip_thresh

  # clip the negative part of sig_in:
  neg_clip_indices = sig_in < -clip_thresh
  sig_out[neg_clip_indices] = -clip_thresh

  # make up attenuation due to clipping. This is a little more robust than
  # the method described in the text above, but functionally equivalent:
  sig_out = (sig_out / np.max(np.abs(sig_out))) # a little less than 32768.

  return sig_out

samplerate = 44100; fs = 100
t = np.linspace(0., 1, samplerate)
amplitude = 1
sine_data = amplitude * np.sin(2. * np.pi * fs * t)

hard_clip_mu_75 = hard_clipper_makeup(sine_data,0.75)
hard_clip_mu_50 = hard_clipper_makeup(sine_data,0.50)
hard_clip_mu_25 = hard_clipper_makeup(sine_data,0.25)

fig, ax = plt.subplots(1,1,figsize=(20, 10))
T = 1000
ax.plot(t[0:T], sine_data[0:T],label='thresh = 1.0')
ax.plot(t[0:T], hard_clip_mu_75[0:T],label='thresh = 0.75')
ax.plot(t[0:T], hard_clip_mu_50[0:T],label='thresh = 0.50')
ax.plot(t[0:T], hard_clip_mu_25[0:T],label='thresh = 0.25')

ax.set_ylabel("Amplitude")
ax.grid(True)
ax.legend()
ax.set_xlabel("Time [s]")
plt.show()

### Visualizing hard-clipping in the frequency domain
Although clipping alone is not soley responsible for the original distorted guitar sound, it does explain one thing we saw earlier: the high frequency content in the distorted guitar recording.

Let's explore that a little more closely, using a signal analysis tool called the [Fast Fourier Transform](https://en.wikipedia.org/wiki/Fast_Fourier_transform) (FFT). This is a numerical algorithm for finding the frequency content of a signal, and it is based on the idea that an arbitrary periodic signal can be approximated by an infinite sum of sines and cosines.

First we'll use SciPy to take the FFT of our sine wave and the various hard-clipped versions:

In [ ]:
# based on https://realpython.com/python-scipy-fft/

from scipy.fft import rfft, rfftfreq

# Number of samples:
N = int(samplerate * t[-1]) # cast to int b/c required by rfftfreq.

fmax = int(1e3)
xf = [[],[],[],[]]
yf = [[],[],[],[]]

# take the FFT of the unclipped sine wave:
yf[0] = rfft(sine_data)
xf[0] = rfftfreq(N, 1/samplerate)

# FFT of lightly-clipped sine wave:
yf[1] = rfft(hard_clip_mu_75)
xf[1] = rfftfreq(N, 1/samplerate)

# FFT of moderately-clipped sine wave:
yf[2] = rfft(hard_clip_mu_50)
xf[2] = rfftfreq(N, 1/samplerate)

# FFT of heavily-clipped sine wave:
yf[3] = rfft(hard_clip_mu_25)
xf[3] = rfftfreq(N, 1/samplerate)

fig, axs = plt.subplots(2,2,figsize=(20,20))
axs[0,0].plot(xf[0][0:fmax], np.abs(yf[0][0:fmax])*(2/N))
axs[0,0].set_xlabel('Frequency [Hz]')
axs[0,0].set_ylabel('Amplitude')
axs[0,0].set_title('FFT, unclipped signal')

axs[0,1].plot(xf[1][0:fmax], np.abs(yf[1][0:fmax])*(2/N))
axs[0,1].set_xlabel('Frequency [Hz]')
axs[0,1].set_ylabel('Amplitude')
axs[0,1].set_title('FFT, lightly-clipped signal')

axs[1,0].plot(xf[2][0:fmax], np.abs(yf[2][0:fmax])*(2/N))
axs[1,0].set_xlabel('Frequency [Hz]')
axs[1,0].set_ylabel('Amplitude')
axs[1,0].set_title('FFT, moderately-clipped signal')

axs[1,1].plot(xf[3][0:fmax], np.abs(yf[3][0:fmax])*(2/N))
axs[1,1].set_xlabel('Frequency [Hz]')
axs[1,1].set_ylabel('Amplitude')
axs[1,1].set_title('FFT, heavily-clipped signal')

plt.show()

Let's try our hard-clipper (with gain makeup) on `riff_clean.wav` and see how it sounds. Don't get your hopes up!

In [ ]:
riff_hc75_data = hard_clipper_makeup(riff_clean_dict['wav_data'],0.75) # sounds almost like the clean signal
sf.write('riff_hc75.wav',riff_hc75_data,44100)
riff_hc75_dict = {'wav_data':riff_hc75_data,
                  'samplerate':44100,
                  'wav_fname':'riff_hc75.wav'}
plot_wave(riff_hc75_dict)

riff_hc50_data = hard_clipper_makeup(riff_clean_dict['wav_data'],0.50) # crunch is barely perceptible
sf.write('riff_hc50.wav',riff_hc50_data,44100)
riff_hc50_dict = {'wav_data':riff_hc50_data,
                  'samplerate':44100,
                  'wav_fname':'riff_hc50.wav'}
plot_wave(riff_hc50_dict)

riff_hc25_data = hard_clipper_makeup(riff_clean_dict['wav_data'],0.25) # crunch begins to be noticeable, still pretty clean
sf.write('riff_hc25.wav',riff_hc25_data,44100)
riff_hc25_dict = {'wav_data':riff_hc25_data,
                  'samplerate':44100,
                  'wav_fname':'riff_hc25.wav'}
plot_wave(riff_hc25_dict)

riff_hc10_data = hard_clipper_makeup(riff_clean_dict['wav_data'],0.10) # now it's starting to really distort
sf.write('riff_hc10.wav',riff_hc10_data,44100)
riff_hc10_dict = {'wav_data':riff_hc10_data,
                  'samplerate':44100,
                  'wav_fname':'riff_hc10.wav'}
plot_wave(riff_hc10_dict)

riff_hc05_data = hard_clipper_makeup(riff_clean_dict['wav_data'],0.05) # now it's starting to really distort
sf.write('riff_hc05.wav',riff_hc05_data,44100)
riff_hc05_dict = {'wav_data':riff_hc05_data,
                  'samplerate':44100,
                  'wav_fname':'riff_hc05.wav'}
plot_wave(riff_hc05_dict)

In [ ]:
Audio(filename='riff_hc75.wav')

In [ ]:
Audio(filename='riff_hc50.wav')

In [ ]:
Audio(filename='riff_hc25.wav')

In [ ]:
Audio(filename='riff_hc10.wav')

In [ ]:
Audio(filename='riff_hc05.wav')

## Bandpass filtering, before clipping stage
We're making good progress toward reverse-engineering the original distorted guitar sound. For comparison, here it is again:

In [ ]:
Audio(filename='riff_dist.wav')

Oh. Hm. We're not so close after all. If it's any comfort, I faced this same exact problem when I was building a hardware distortion pedal based on the ProCo RAT circuit: it distorted just fine, but it sounded "fuzzy" and not, well, metal.

Here's the current problem:
* in the absence of any filtering before the clipping stage, the low-frequency signal content from my guitar dominates the clipping stage and makes the clipper's output sound "fuzzy", like a blown speaker.
* I can't keep adding more distortion (cranking up the gain on the input to the clipper, or equivalently, reducing the clipping threshold and then adding makeup gain).
* in other words, I want more distortion, but not at the low frequencies

How did I achieve that? Filtering! Specifically, bandpass filtering. Here's why that helps:
* First, I filter out low-frequency content before the clipping stage
* Then, I really crank up the distortion at the clipping stage, by reducing the clipping threshold
* As a result, the mid-range frequencies distort while the bottom-end is attenuated

I didn't exactly come up with this myself. The Boss Metal Zone pedal [does this exact same thing](https://electricdruid.net/boss-mt-2-metal-zone-pedal-analysis/), albeit much more cleverly.

### Complementary Filters
So, now we come to a question: how does one implement a filter in Python? This is potentially a bottomless rabbit hole, but before diving in, let's look at a really simple approach to digital filters: the _**complementary filter**_. This is basically a weighted average.

Consider an input signal $x$ and an output signal $y$. These are discrete-time signals, so they can be written as $x[i], y[i],\  i = 1,2,\ldots,n$. It often makes sense to think of $x[n]$ and $y[n]$ as the "current" input and output, $x[n-1]$ and $y[n-1]$ as the "previous" input and output, and so on.

The next two equations use scalar gains $a$ and $b$ which are required to be between 0 and 1.

There are two basic kinds of complementary filters. They are recursive, because they process an input signal as well as their past outputs. These filters belong to a larger family of filters called "infinite-impulse-response" (IIR) filters. As you might guess, there is a separate family of "finite-impulse-response" (FIR) filters. Each filter architecture has pros and cons.

The next two examples demonstrate two complementary filter equations.

#### Complementary lowpass filter
The block diagram of a complementary lowpass filter is
<img src="https://raw.githubusercontent.com/dlynch7/reverse_engineer_guitar_fx/main/distortion/LPF.png" alt="LPF.png" width="600"/>
If you haven't seen these kinds of block diagrams before, don't worry. They're pretty straightforward. The one symbol you might find unfamiliar is the triangle with numbers/letters inside it: that represents a "gain" stage, where the output signal's amplitude is some multiple or fraction of the input signal's amplitude, depending on the value of the number/letter inside the triangle.

The equation for a complementary highpass filter is
\begin{equation}
y[n]=(1-a)x[n] + a y[n-1]
\end{equation}

Think of this as a weighted average between the current input and the previous output. Consider two limits:
* As $a\rightarrow 0$, $y[n] \rightarrow x[n]$, in which case the output is simply the input, and no filtering occurs; all input signal content is passed through.
* As $a\rightarrow 1$, $y[n] \rightarrow y[n-1]$, in which case the output doesn't change in response to input; the signal is maximally lowpass filtered because only the DC component remains.

In [ ]:
def lowpass_iir(sig_in,filt_coef):
    if not type(sig_in) is np.ndarray:
      raise TypeError('type(sig_in) must be numpy.ndarray')
    if not (type(filt_coef) is float or type(filt_coef) is int):
      raise TypeError('type(filt_coef) must be float or int')
    if (filt_coef > 1 or filt_coef < 0):
      raise ValueError('filt_coef must be between 0 and 1.')
    
    sig_out = np.zeros_like(sig_in)
#     print(sig_out.shape)
    
    for n in range(1,sig_in.shape[0]): # notice that we're starting at "1" instead of "0"
        sig_out[n] = (1-filt_coef)*sig_in[n] + filt_coef*sig_out[n-1]
    
    return sig_out

# To test our lowpass filter, create a sinusoidal signal and then hard-clip it:
samplerate = 44100; fs = 100
t = np.linspace(0., 1, samplerate)
amplitude = 1
sine_data = amplitude * np.sin(2. * np.pi * fs * t)
hard_clip_mu_10 = hard_clipper_makeup(sine_data,0.10) # this will look like a square wave: lots of harmonic content!

# Now let's lowpass-filter our quasi-square wave:
lpf_square_wave_50 = lowpass_iir(hard_clip_mu_10,0.5) # set the filter coefficient to 0.5
lpf_square_wave_75 = lowpass_iir(hard_clip_mu_10,0.75) # set the filter coefficient to 0.75
lpf_square_wave_90 = lowpass_iir(hard_clip_mu_10,0.90) # set the filter coefficient to 0.90
lpf_square_wave_95 = lowpass_iir(hard_clip_mu_10,0.95) # set the filter coefficient to 0.90

fig, ax = plt.subplots(1,1,figsize=(20, 10))
T = 1000
ax.plot(t[0:T], sine_data[0:T],label='pure sine')
ax.plot(t[0:T], hard_clip_mu_10[0:T],label='in: hard-clipped sine, thresh = 0.10')
ax.plot(t[0:T], lpf_square_wave_50[0:T],label="out: LPF'd sine, coeff = 0.5")
ax.plot(t[0:T], lpf_square_wave_75[0:T],label="out: LPF'd sine, coeff = 0.75")
ax.plot(t[0:T], lpf_square_wave_90[0:T],label="out: LPF'd sine, coeff = 0.90")
ax.plot(t[0:T], lpf_square_wave_95[0:T],label="out: LPF'd sine, coeff = 0.95")

ax.set_ylabel("Amplitude")
ax.grid(True)
ax.legend()
ax.set_xlabel("Time [s]")
plt.show()

We just _saw_ what a complementary _**lowpass**_ filter does to a signal. Now let's _hear_ what it does to our heavily-clipped guitar recording.

Before lowpass filtering:

In [ ]:
Audio(filename='riff_hc05.wav')

After lowpass filtering:

In [ ]:
sf.write('riff_hc05_lpf.wav',
         lowpass_iir(riff_hc05_data,0.75),
         44100)

Audio(filename='riff_hc05_lpf.wav')

Sounds muffled, right? That means our lowpass filter is working!

Now let's move on and implement a complementary _**highpass**_ filter.

#### Complementary highpass filter
The block diagram of a complementary highpass filter is
<img src="https://raw.githubusercontent.com/dlynch7/reverse_engineer_guitar_fx/main/distortion/HPF.png" alt="HPF.png" width="500"/>

The equation for a complementary highpass filter is
\begin{equation}
y[n]=(1-b) x[n] + b(x[n]-x[n-1])
\end{equation}

Again, think of this as a weighted average between the current input and the previous output, and again consider two limits:
* As $b\rightarrow 0$, $y[n] \rightarrow x[n]$, in which case the output is simply the input, and no filtering occurs; all input signal content is passed through.
* As $b\rightarrow 1$, $y[n] \rightarrow x[n]-x[n-1]$, in which case the output is maximally highpass filtered, as it tracks the _difference_ between the current input and the previous input, thereby approximating the continuous-time derivative of the signal.

In [ ]:
def highpass_iir(sig_in,filt_coef):
    if not type(sig_in) is np.ndarray:
      raise TypeError('type(sig_in) must be numpy.ndarray')
    if not (type(filt_coef) is float or type(filt_coef) is int):
      raise TypeError('type(filt_coef) must be float or int')
    if (filt_coef > 1 or filt_coef < 0):
      raise ValueError('filt_coef must be between 0 and 1.')
    
    sig_out = np.zeros_like(sig_in)
#     print(sig_out.shape)
    
    for n in range(1,sig_in.shape[0]): # notice that we're starting at "1" instead of "0"
        sig_out[n] = (1-filt_coef)*sig_in[n] + filt_coef*(sig_in[n] - sig_in[n-1])
    
    return sig_out

# To test our highpass filter, create a sinusoidal signal and then hard-clip it:
samplerate = 44100; fs = 100
t = np.linspace(0., 1, samplerate)
amplitude = 1
sine_data = amplitude * np.sin(2. * np.pi * fs * t)
hard_clip_mu_10 = hard_clipper_makeup(sine_data,0.10) # this will look like a square wave: lots of harmonic content!

# Now let's highpass-filter our quasi-square wave:
hpf_square_wave_50 = highpass_iir(hard_clip_mu_10,0.5) # set the filter coefficient to 0.5
hpf_square_wave_75 = highpass_iir(hard_clip_mu_10,0.75) # set the filter coefficient to 0.75
hpf_square_wave_90 = highpass_iir(hard_clip_mu_10,0.90) # set the filter coefficient to 0.90
hpf_square_wave_95 = highpass_iir(hard_clip_mu_10,0.95) # set the filter coefficient to 0.90

fig, ax = plt.subplots(1,1,figsize=(20, 10))
T = 1000
ax.plot(t[0:T], sine_data[0:T],label='pure sine')
ax.plot(t[0:T], hard_clip_mu_10[0:T],label='in: hard-clipped sine, thresh = 0.10')
ax.plot(t[0:T], hpf_square_wave_50[0:T],label="out: HPF'd sine, coeff = 0.5")
ax.plot(t[0:T], hpf_square_wave_75[0:T],label="out: HPF'd sine, coeff = 0.75")
ax.plot(t[0:T], hpf_square_wave_90[0:T],label="out: HPF'd sine, coeff = 0.90")
ax.plot(t[0:T], hpf_square_wave_95[0:T],label="out: HPF'd sine, coeff = 0.95")

ax.set_ylabel("Amplitude")
ax.grid(True)
ax.legend()
ax.set_xlabel("Time [s]")
plt.show()

Now let's hear what it sounds like, again applied to our hard-clipped guitar signal. First, here's the un-filtered recording:

In [ ]:
Audio(filename='riff_hc05.wav')

And now, the highpass-filtered version:

In [ ]:
sf.write('riff_hc05_hpf.wav',
         highpass_iir(riff_hc05_data,0.90),
         44100)
Audio(filename='riff_hc05_hpf.wav')

Sounds tinnier, right? That means our complementary highpass filter is working.

#### Complementary bandpass filter
We can simply cascade the lowpass and highpass filters:
<img src="https://raw.githubusercontent.com/dlynch7/reverse_engineer_guitar_fx/main/distortion/BPF.png" alt="BPF.png" width="400"/>
Mathematically,
\begin{align}
w[n]=(1-a)x[n] + a w[n-1],\\
y[n]=(1-b)w[n] + b(w[n]-w[n-1]),
\end{align}

Again, consider the limit cases:
* As $a,b\rightarrow 0$, $y[n] \rightarrow x[n]$ (pass-through);
* Hold $a=0$; as $b\rightarrow 1$, $y[n] \rightarrow x[n]-x[n-1]$ (highpass);
* Hold $b=0$; as $a\rightarrow 1$, $y[n] \rightarrow y[n-1]$ (lowpass);
* As $a,b\rightarrow 1$; the cascaded filters work as a bandpass filter, letting mid-frequency content pass through while attenuating low-frequency and high-frequency content. If you let $a < b$, some frequency range of the input signal will be unattenuated.

Now, let's implement this in a function. If we were writing a real-time audio application, we would have to write this in a way that works with streaming input and output, but instead, we'll write our function to post-process an existing signal:

In [ ]:
def bandpass_iir(sig_in,lpf_coef,hpf_coef):
    # this function just calls other functions we wrote,
    # and those functions check their inputs, so we don't need to do that here.
    
    # first, lowpass-filter the input signal
    sig_med = lowpass_iir(sig_in,lpf_coef)
    
    # second, highpass-filter the lowpass-filtered signal
    sig_out = highpass_iir(sig_med,hpf_coef)
    
    return sig_out

# To test our bandpass filter, create a sinusoidal signal and then hard-clip it:
samplerate = 44100; fs = 100
t = np.linspace(0., 1, samplerate)
amplitude = 1
sine_data = amplitude * np.sin(2. * np.pi * fs * t)
hard_clip_mu_10 = hard_clipper_makeup(sine_data,0.10) # this will look like a square wave: lots of harmonic content!

# Now let's highpass-filter our quasi-square wave:
bpf_square_wave_10_10 = bandpass_iir(hard_clip_mu_10,0.1,0.1) # LPF coefficient = HPF coefficient = 0.1
bpf_square_wave_10_20 = bandpass_iir(hard_clip_mu_10,0.1,0.2) # LPF coef = 0.1, HPF coef = 0.2
bpf_square_wave_20_50 = bandpass_iir(hard_clip_mu_10,0.2,0.5) # LPF coef = 0.2, HPF coef = 0.5
bpf_square_wave_50_90 = bandpass_iir(hard_clip_mu_10,0.5,0.9) # LPF coef = 0.5, HPF coef = 0.9

fig, ax = plt.subplots(1,1,figsize=(20, 10))
T = 1000
ax.plot(t[0:T], sine_data[0:T],label='pure sine')
ax.plot(t[0:T], hard_clip_mu_10[0:T],label='in: hard-clipped sine, thresh = 0.10')
ax.plot(t[0:T], bpf_square_wave_10_10[0:T],label="out: BPF'd sine, coeffs = (0.1,0.1)")
ax.plot(t[0:T], bpf_square_wave_10_20[0:T],label="out: BPF'd sine, coeffs = (0.1,0.2)")
ax.plot(t[0:T], bpf_square_wave_20_50[0:T],label="out: BPF'd sine, coeffs = (0.2,0.5)")
ax.plot(t[0:T], bpf_square_wave_50_90[0:T],label="out: BPF'd sine, coeffs = (0.5,0.9)")

ax.set_ylabel("Amplitude")
ax.grid(True)
ax.legend()
ax.set_xlabel("Time [s]")
plt.show()

As usual, let's test it out on our guitar recording. Here's the unfiltered recording:

In [ ]:
Audio(filename='riff_hc05.wav')

And here's a bandpass-filtered version:

In [ ]:
sf.write('riff_hc05_bpf.wav',
         bandpass_iir(riff_hc05_data,0.75,0.9),
         44100)

Audio(filename='riff_hc05_bpf.wav')

For comparison, here's the lowpass-filtered version:

In [ ]:
Audio(filename='riff_hc05_lpf.wav')

And also for comparison, here's the highpass-filtered version:

In [ ]:
Audio(filename='riff_hc05_hpf.wav')

## Cascade # 1: bandpass filter, then clip
Now let's start putting things together.
1. First, we'll bandpass-filter the incoming signal;
2. Then, we'll send the filtered signal through our clipping stage (with makeup gain).

Remember why we're going this:
* by filtering out some of the low-frequency content from the clean guitar signal, we can apply more distortion via clipping before the output starts to sound bad
* there isn't much frequency content above a few kHz coming from the guitar, so we might as well filter out what little is there.

In [ ]:
def bpf_and_clip(sig_in,lpf_coef,hpf_coef,clip_thresh):
    # this function just calls the other functions we wrote,
    # and they check their inputs, so we don't need to do that here.
    
    # First, bandpass-filter the input signal:
    sig_med = bandpass_iir(sig_in,lpf_coef,hpf_coef)
    
    # Second, hard-clip the bandpass-filtered signal:
    sig_out = hard_clipper_makeup(sig_med,clip_thresh)
    
    return sig_out

# test it out on audio directly:
lpf_coef = 0.75
hpf_coef = 0.99
clip_thresh = 0.001
samplerate = 44100

sf.write('riff_bpf_hc.wav',
         bpf_and_clip(riff_clean_data,lpf_coef,hpf_coef,clip_thresh),
         samplerate)

Audio(filename='riff_bpf_hc.wav')

As usual, here's our target sound, for comparison:

In [ ]:
Audio(filename='riff_dist.wav')

## Cascade #2: BPF, then clip, then BPF
Now it's getting pretty close, but our reverse-engineered signal chain still sounds harsh on the ears, and I think some additional filtering after the clipping stage will help with this.

This is also what is done in the ProCo RAT, the Boss MT-2 Metal Zone, and many other distortion units.

The overarching goal is to pack as much distortion into our effects as possible without it sounding "crappy", and smoothing out the clipper's output will help with that. Let's give it a shot!

In [ ]:
def bpf_clip_bpf(sig_in,lpf_coef1,hpf_coef1,clip_thresh,lpf_coef2,hpf_coef2):
    # this function just calls the other functions we wrote,
    # and they check their inputs, so we don't need to do that here.
    
    # First, bandpass-filter the input signal:
    sig_1 = bandpass_iir(sig_in,lpf_coef,hpf_coef)
    
    # Second, hard-clip the bandpass-filtered signal:
    sig_2 = hard_clipper_makeup(sig_1,clip_thresh)
    
    # Third, lowpass-filter the clipped signal:
    sig_out = bandpass_iir(sig_2,lpf_coef2,hpf_coef2)
    
    return sig_out

Here's the clean guitar signal, which we're going to distort the heck out of:

In [ ]:
Audio(filename='riff_clean.wav')

Now let's use the function we just wrote to process the clean signal:

In [ ]:
lpf_coef1 = 0.5
hpf_coef1 = 0.999
clip_thresh = 0.0001
lpf_coef2 = 0.875
hpf_coef2 = 0.6
samplerate = 44100

sf.write('riff_bpf_hc_bpf.wav',
         bpf_clip_bpf(riff_clean_data,lpf_coef1,hpf_coef1,clip_thresh,lpf_coef2,hpf_coef2),
         samplerate)

Audio(filename='riff_bpf_hc_bpf.wav')

As usual, here's our target sound, for comparison:

In [ ]:
Audio(filename='riff_dist.wav')

### FFTs of the guitar signal at each stage in the effects chain

In [ ]:
# based on https://realpython.com/python-scipy-fft/

from scipy.fft import rfft, rfftfreq

lpf_coef1 = 0.5
hpf_coef1 = 0.999
clip_thresh = 0.0001
lpf_coef2 = 0.9 # 0.85
hpf_coef2 = 0.7

# Number of samples:
samplerate = 44100
N = samplerate # N is the number of samples, and (below) we're going to restrict ourselves to 1 second of data

fmax = int(2e4)
xf = [[],[],[],[]]
yf = [[],[],[],[]]
yf_db = [[],[],[],[]]

# take the FFT of the clean guitar signal:
yf[0] = rfft(riff_clean_data[0:N][:,0])
yf_db[0] = 20 * np.log10(np.abs(yf[0] / yf[0][0]))
xf[0] = rfftfreq(N, 1/samplerate)

# FFT of bandpass-filtered guitar signal:
bpf_data = bpf_clip_bpf(riff_clean_data,lpf_coef1,hpf_coef1,1,0,0)
yf[1] = rfft(bpf_data[0:N][:,0])
yf_db[1] = 20 * np.log10(np.abs(yf[1] / yf[0][0]))
xf[1] = rfftfreq(N, 1/samplerate)

# FFT of bandpass-filtered and hard-clipped guitar signal:
bpf_hc_data = bpf_clip_bpf(riff_clean_data,lpf_coef1,hpf_coef1,clip_thresh,0,0)
yf[2] = rfft(bpf_hc_data[0:N][:,0])
yf_db[2] = 20 * np.log10(np.abs(yf[2] / yf[0][0]))
xf[2] = rfftfreq(N, 1/samplerate)

# FFT of bandpass-filtered, hard-clipped, then re-bandpass-filtered guitar signal:
bpf_hc_bpf_data = bpf_clip_bpf(riff_clean_data,lpf_coef1,hpf_coef1,clip_thresh,lpf_coef2,hpf_coef2)
yf[3] = rfft(bpf_hc_bpf_data[0:N][:,0])
yf_db[3] = 20 * np.log10(np.abs(yf[3] / yf[0][0]))
xf[3] = rfftfreq(N, 1/samplerate)

fig, axs = plt.subplots(2,2,figsize=(20,20))
axs[0,0].plot(xf[0][0:fmax], yf_db[0][0:fmax])
axs[0,0].set_xlabel('Frequency [Hz]')
axs[0,0].set_xscale('log')
axs[0,0].set_ylabel('Amplitude [dB]')
axs[0,0].set_title('FFT, clean signal')

axs[0,1].plot(xf[1][0:fmax], yf_db[1][0:fmax])
axs[0,1].set_xlabel('Frequency [Hz]')
axs[0,1].set_xscale('log')
axs[0,1].set_ylabel('Amplitude [dB]')
axs[0,1].set_title("FFT, BPF'd signal")

axs[1,0].plot(xf[2][0:fmax], yf_db[2][0:fmax])
axs[1,0].set_xlabel('Frequency [Hz]')
axs[1,0].set_xscale('log')
axs[1,0].set_ylabel('Amplitude [dB]')
axs[1,0].set_title("FFT, BPF'd & hard-clipped signal")

axs[1,1].plot(xf[3][0:fmax], yf_db[3][0:fmax])
axs[1,1].set_xlabel('Frequency [Hz]')
axs[1,1].set_xscale('log')
axs[1,1].set_ylabel('Amplitude [dB]')
axs[1,1].set_title("FFT, BPF'd, hard-clipped, re-BPF'd signal")

plt.show()

## Conclusion
We managed to get a decent approximation of the distorted guitar recording, using a cascade of pretty simple effects units:
* bandpass filter (itself implemented as a cascade: a lowpass complementary filter followed by a highpass complementary filter)
* hard-clipping followed by makeup gain
* another bandpass filter to smooth the sonic rough edges introduced by hard-clipping

Still, our reverse-engineered effects chain doesn't quite sound like the original recording, and there are a number of things that could be done to get closer. Three extensions that stand out are
* apply some "compression" (aka dynamic gain control) to the clean input before filtering it, clipping it, etc. This will provide a more uniform signal to the effects later in the signal chain, and that could be advantageous;
* replace the hard-clipping effect with a soft-clipping effect: instead of simply saturating the input signal, a smooth function like a logistic curve or the arctangent could be used to achieve the desired distortion with less of a "bitcrushed" sound;
* put more effort into filter design, starting with replacing the complementary filters with finite-impulse-response filters, which introduce fewer audio artifacts and whose frequency-domain properties can be designed using SciPy tools.